In [1]:
# ! pip install openpyxl pandas numpy

In [2]:
import pandas as pd
import numpy as np

channels = pd.read_excel("../data/tg_channels.xlsx")
#df = pd.read_csv("../data/cleaned_news_exp.csv")[["message_id", "id_channel", "message", "date", "topic"]]
df = pd.read_parquet("../data/tg_news_full.parquet")[["message_id", "id_channel", "message", "date"]]

/home/mlcore/conda/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
def attach_channel_w(
    df: pd.DataFrame,
    channels_df: pd.DataFrame,
    news_id_col: str = "id_channel",
    chan_id_col: str = "id",
    subs_col: str = "subscribers",
    weight_col: str = "channel_w",) -> pd.DataFrame:
    ch = channels_df[[chan_id_col, subs_col]].copy()
    ch[chan_id_col] = pd.to_numeric(ch[chan_id_col], errors="coerce").astype("Int64")
    ch[subs_col] = pd.to_numeric(ch[subs_col], errors="coerce").fillna(0).astype(float)
    id2subs = dict(zip(ch[chan_id_col], ch[subs_col]))

    out = df.copy()
    out[news_id_col] = pd.to_numeric(out[news_id_col], errors="coerce").astype("Int64")

    subs = out[news_id_col].map(id2subs).fillna(0.0).to_numpy(dtype=np.float32)
    log_subs = np.log1p(subs)

    mn, mx = float(log_subs.min()), float(log_subs.max())
    if mx > mn:
        w = (log_subs - mn) / (mx - mn)
    else:
        w = np.zeros_like(log_subs, dtype=np.float32)

    out[weight_col] = w.astype(np.float32)
    return out


In [4]:
ch_map = (channels[["id", "name"]]
          .dropna()
          .assign(id=lambda x: pd.to_numeric(x["id"], errors="coerce"))
          .dropna(subset=["id"])
          .assign(id=lambda x: x["id"].astype(int))
          .set_index("id")["name"]
          .to_dict())

df = df.copy()
df["id_channel"] = pd.to_numeric(df["id_channel"], errors="coerce")
df["channel_name"] = df["id_channel"].map(ch_map).fillna(df["id_channel"].astype("Int64").astype(str))
df = attach_channel_w(df, channels)

In [5]:
import re
import numpy as np
import pandas as pd

def clean_news_text(t: str) -> str:
    t = t or ""
    t = re.sub(r"#\w+", " ", t)
    t = re.sub(r"[⚡️📈📉🇷🇺✅❗️🔥⬛ ⬜ ⚫ ⚪🔹]+", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

def ensure_datetime(df: pd.DataFrame, col: str = "date") -> pd.DataFrame:
    d = df.copy()
    d[col] = pd.to_datetime(d[col], utc=True, errors="coerce")
    d = d.dropna(subset=[col])
    d["date_day"] = d[col].dt.floor("D")
    return d

df = ensure_datetime(df, "date")
df["message_id"] = df["message_id"].astype(str)
df["message"] = df["message"].fillna("").astype(str).map(clean_news_text)

In [6]:
df.head()

,message_id,id_channel,message,date,channel_name,channel_w,date_day
0,275548,3,"Правительство Словакии обсудит меры, которые м...",2025-01-02 17:00:02+00:00,Риа Новости,1.0,2025-01-02 00:00:00+00:00
1,275547,3,В ДТП с туристическим автобусом в Таиланде пос...,2025-01-02 16:40:53+00:00,Риа Новости,1.0,2025-01-02 00:00:00+00:00
2,275546,3,Премьер Израиля Нетаньяху выписан из больницы ...,2025-01-02 16:20:12+00:00,Риа Новости,1.0,2025-01-02 00:00:00+00:00
3,275545,3,Подозреваемый в подрыве автомобиля Tesla Cyber...,2025-01-02 15:54:29+00:00,Риа Новости,1.0,2025-01-02 00:00:00+00:00
4,275543,3,Спецоперация. Обстановка и главные события на ...,2025-01-02 15:32:55+00:00,Риа Новости,1.0,2025-01-02 00:00:00+00:00


In [7]:
#! pip install rank_bm25

In [8]:
from rank_bm25 import BM25Okapi

def tokenize_ru(text: str):
    text = text.lower()
    text = re.sub(r"[^0-9a-zа-яё\s]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.split()

corpus_tok = [tokenize_ru(t) for t in df["message"].tolist()]
bm25 = BM25Okapi(corpus_tok)

In [9]:
#! pip install faiss-cpu faiss-gpu-cu12 faiss-gpu-cu11

In [10]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/multilingual-e5-large" 
encoder = SentenceTransformer(MODEL_NAME, device="cuda")  
texts = df["message"].tolist()
doc_inputs = ["passage: " + t for t in texts]

In [11]:
# E_docs = encoder.encode(
#     doc_inputs,
#     batch_size=64,
#     show_progress_bar=True,
#     normalize_embeddings=True,
# ).astype(np.float32)

In [12]:
# import faiss

# dim = E_docs.shape[1]
# index = faiss.IndexFlatIP(dim)      
# index.add(E_docs)

In [13]:
# # save
# from pathlib import Path
# import numpy as np
# import pandas as pd
# import faiss
# import pickle

# OUT = Path("indexes")
# OUT.mkdir(parents=True, exist_ok=True)

# #rowmap
# rowmap = df[["message_id","date","date_day","id_channel","channel_name"]].copy()
# rowmap.to_parquet(OUT / "rowmap.parquet", index=False)

# np.save(OUT / "E_docs_e5_large.npy", E_docs)

# faiss.write_index(index, str(OUT / "faiss_e5_large.index"))

# with open(OUT / "bm25_corpus_tok.pkl", "wb") as f:
#     pickle.dump(corpus_tok, f)


In [14]:
# load
from pathlib import Path
import numpy as np
import pandas as pd
import faiss
import pickle

INP = Path("indexes")

rowmap = pd.read_parquet(INP / "rowmap.parquet")

E_docs = np.load(INP / "E_docs_e5_large.npy")
index = faiss.read_index(str(INP / "faiss_e5_large.index"))

with open(INP / "bm25_corpus_tok.pkl", "rb") as f:
    corpus_tok = pickle.load(f)

from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(corpus_tok)


In [15]:
assert len(rowmap) == E_docs.shape[0]
assert len(rowmap) == index.ntotal

In [16]:
import re
import hashlib
import numpy as np
import pandas as pd

try:
    import faiss
except Exception:
    faiss = None

_URL_RE = re.compile(r"https?://\S+|www\.\S+")
_HANDLE_RE = re.compile(r"@\w+")
_WS_RE = re.compile(r"\s+")

def _normalize_for_dedup(text: str, mask_numbers: bool = True) -> str:
    if not isinstance(text, str):
        return ""
    t = text.lower()
    t = _URL_RE.sub(" ", t)
    t = _HANDLE_RE.sub(" ", t)
    t = re.sub(r"[^\w\s%.,\-]+", " ", t, flags=re.UNICODE)
    if mask_numbers:
        t = re.sub(r"\d+(?:[.,]\d+)?", "<num>", t)
    t = _WS_RE.sub(" ", t).strip()
    return t

def _stable_hash(s: str) -> str:
    return hashlib.md5(s.encode("utf-8", errors="ignore")).hexdigest()

def _union_find(n: int):
    parent = np.arange(n, dtype=np.int32)
    rank = np.zeros(n, dtype=np.int8)
    def find(x: int) -> int:
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return int(x)
    def union(a: int, b: int) -> None:
        ra, rb = find(a), find(b)
        if ra == rb:
            return
        if rank[ra] < rank[rb]:
            parent[ra] = rb
        elif rank[ra] > rank[rb]:
            parent[rb] = ra
        else:
            parent[rb] = ra
            rank[ra] += 1
    return find, union

def dedup_cluster_candidates_time(
    cand: pd.DataFrame,
    encoder,
    text_col: str = "message",
    date_col: str = "date_day",
    channel_col: str = "channel_name",
    score_col: str = "score_rrf",
    sim_threshold: float = 0.95,
    knn: int = 20,
    keep_per_cluster: int = 1,
    mask_numbers: bool = True,
    max_day_diff: int = 1,
    overwrite_channel: bool = True,
    channel_join: str = "; ",
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:

    if cand is None or len(cand) == 0:
        return cand, pd.DataFrame(), pd.DataFrame()

    cand = cand.copy().reset_index(drop=True)

    dts = pd.to_datetime(cand[date_col], errors="coerce", utc=True).dt.normalize()
    cand["_dt"] = dts
    cand["_dt_str"] = cand["_dt"].dt.strftime("%Y-%m-%d").fillna("")

    norm = cand[text_col].fillna("").map(lambda s: _normalize_for_dedup(s, mask_numbers=mask_numbers))
    cand["_h"] = norm.map(_stable_hash)

    cand["_hk"] = cand["_h"].astype(str) + "|" + cand["_dt_str"].astype(str)

    if score_col in cand.columns:
        rep_idx = (
            cand.sort_values(score_col, ascending=False)
                .groupby("_hk", as_index=False)
                .head(1)
                .index.to_numpy()
        )
    else:
        rep_idx = cand.groupby("_hk", as_index=False).head(1).index.to_numpy()

    rep = cand.loc[rep_idx].copy().reset_index(drop=True)

    texts = rep[text_col].fillna("").tolist()
    X = encoder.encode(texts, normalize_embeddings=True, show_progress_bar=False).astype(np.float32)

    rep_dt = pd.to_datetime(rep["_dt"], errors="coerce", utc=True)
    rep_dt = rep_dt.dt.tz_convert(None).dt.normalize().to_numpy(dtype="datetime64[D]")

    m = len(rep)
    find, union = _union_find(m)

    if m > 1:
        if faiss is None:
            S = X @ X.T
            for i in range(m):
                js = np.where(S[i, i+1:] >= sim_threshold)[0] + (i + 1)
                for j in js:
                    if np.isnat(rep_dt[i]) or np.isnat(rep_dt[j]):
                        continue
                    day_diff = abs(int((rep_dt[i] - rep_dt[j]).astype("timedelta64[D]").astype(int)))
                    if day_diff <= max_day_diff:
                        union(i, int(j))
        else:
            idx = faiss.IndexFlatIP(X.shape[1])
            idx.add(X)
            D, I = idx.search(X, min(knn, m))
            for i in range(m):
                for score, j in zip(D[i], I[i]):
                    if j < 0 or j == i:
                        continue
                    if float(score) < sim_threshold:
                        continue
                    if np.isnat(rep_dt[i]) or np.isnat(rep_dt[j]):
                        continue
                    day_diff = abs(int((rep_dt[i] - rep_dt[j]).astype("timedelta64[D]").astype(int)))
                    if day_diff <= max_day_diff:
                        union(i, int(j))

    rep_cluster = np.array([find(i) for i in range(m)], dtype=np.int32)
    _, rep_cluster = np.unique(rep_cluster, return_inverse=True)
    rep["_rep_cluster"] = rep_cluster

    hk_to_cluster = dict(zip(rep["_hk"].tolist(), rep["_rep_cluster"].tolist()))
    cand["_cluster_id"] = cand["_hk"].map(hk_to_cluster)

    cand["_cluster_id"] = cand["_cluster_id"].fillna(-1).astype(np.int32)

    cluster_sizes = cand.groupby("_cluster_id").size()

    if channel_col in cand.columns:
        ch_joined = (
            cand.groupby("_cluster_id")[channel_col]
                .apply(lambda s: channel_join.join(sorted({str(x) for x in s.dropna().tolist()})))
        )
    else:
        ch_joined = pd.Series(dtype=str)

    if score_col in cand.columns:
        cand_dedup = (
            cand.sort_values(score_col, ascending=False)
                .groupby("_cluster_id", group_keys=False)
                .head(keep_per_cluster)
                .reset_index(drop=True)
        )
    else:
        cand_dedup = (
            cand.groupby("_cluster_id", group_keys=False)
                .head(keep_per_cluster)
                .reset_index(drop=True)
        )

    cand_dedup["cluster_size"] = cand_dedup["_cluster_id"].map(cluster_sizes).astype(int)

    if channel_col in cand.columns:
        cand_dedup["channel_all"] = cand_dedup["_cluster_id"].map(ch_joined).fillna("")
        cand_dedup["channel_primary"] = cand_dedup[channel_col].astype(str)
        if overwrite_channel and channel_col in cand_dedup.columns:
            cand_dedup[channel_col] = cand_dedup["channel_all"]

    cand_dedup = cand_dedup.drop(columns=["_h", "_hk", "_dt", "_dt_str"], errors="ignore")

    clusters = cand[["_cluster_id"]].copy()
    clusters["cluster_size"] = clusters["_cluster_id"].map(cluster_sizes).astype(int)

    members = cand[["_cluster_id"]].copy()
    for c in ["date_day", "date", "channel_name", "channel", "message_id", "score_rrf"]:
        if c in cand.columns:
            members[c] = cand[c]
    members["text_snip"] = cand[text_col].fillna("").map(lambda s: s[:250])

    return cand_dedup, clusters, members

In [17]:
import numpy as np
import pandas as pd

def snippet(t: str, n: int = 1000) -> str:
    return t[:n]

def _topk_indices_from_scores(scores: np.ndarray, k: int) -> np.ndarray:
    k = min(k, len(scores))
    if k <= 0:
        return np.array([], dtype=int)
    if k == len(scores):
        idx = np.argsort(-scores)
    else:
        idx = np.argpartition(-scores, k - 1)[:k]
        idx = idx[np.argsort(-scores[idx])]
    return idx.astype(int)

def dense_candidates_faiss(index, encoder, query: str, topN: int = 500):
    qv = encoder.encode(
        ["query: " + query],
        normalize_embeddings=True,
        show_progress_bar=False
    ).astype(np.float32)
    scores, idx = index.search(qv, topN)
    return idx[0].astype(int), scores[0].astype(np.float32)

def _compute_time_arrays(df: pd.DataFrame, rowpos: np.ndarray, anchor_date, date_col: str):
    ad = pd.to_datetime(anchor_date, utc=True).normalize()
    dts = pd.to_datetime(df.loc[rowpos, date_col], errors="coerce", utc=True).dt.normalize()
    age = (ad - dts).dt.days.to_numpy(dtype=np.float32)
    age = np.where(np.isfinite(age), age, 1e9).astype(np.float32)
    age = np.where(age < 0, 1e9, age).astype(np.float32)
    return dts, age

def _time_rank_from_age(age_days: np.ndarray) -> np.ndarray:
    order = np.argsort(age_days, kind="stable")
    rank = np.empty_like(order, dtype=np.int32)
    rank[order] = np.arange(1, len(order) + 1, dtype=np.int32)
    return rank

def hybrid_retrieve_rrf(
    df: pd.DataFrame,
    index,
    encoder,
    bm25,
    tokenize_fn,
    query: str,
    k: int = 50,
    topN_each: int = 500,
    k_rrf: int = 60,
    w_dense: float = 1.0,
    w_bm25: float = 1.0,
    anchor_date: str | pd.Timestamp | None = None,
    date_col: str = "date_day",
    max_window_days: int | None = 365,
    w_time: float = 0.5,
    w_channel: float | None = None,
    channel_w_col: str = "channel_w",
) -> pd.DataFrame:
    if anchor_date is not None:
        ad = pd.to_datetime(anchor_date, utc=True).normalize()
        if date_col not in df.columns:
            raise KeyError(f"date_col='{date_col}' not found in df.columns")
        dts_all = pd.to_datetime(df[date_col], errors="coerce", utc=True).dt.normalize()
        allowed = (dts_all <= ad)
        if max_window_days is not None:
            age_all = (ad - dts_all).dt.days
            allowed &= (age_all >= 0) & (age_all <= int(max_window_days))
        allowed_np = allowed.to_numpy(dtype=bool)
    else:
        allowed_np = None

    d_idx, _ = dense_candidates_faiss(index, encoder, query, topN=topN_each)
    if allowed_np is not None and len(d_idx) > 0:
        d_idx = d_idx[allowed_np[d_idx]]
    dense_rank = {int(rowpos): r for r, rowpos in enumerate(d_idx, start=1)}

    if bm25 is None:
        union = d_idx.astype(int)
        if len(union) == 0:
            return df.iloc[[]].copy().reset_index(drop=True)

        rrf = w_dense / (k_rrf + np.arange(1, len(union) + 1, dtype=np.float32))

        rank_time = None
        if anchor_date is not None and w_time and len(union) > 0:
            _, age = _compute_time_arrays(df, union, anchor_date, date_col)
            rank_time = _time_rank_from_age(age)
            rrf = rrf + (w_time / (k_rrf + rank_time.astype(np.float32)))

        order = np.argsort(-rrf)
        union = union[order]
        rrf = rrf[order]
        if rank_time is not None:
            rank_time = rank_time[order]

        out = df.iloc[union].copy()
        out["_rowpos"] = union
        out["score_rrf"] = rrf
        out["rank_dense"] = out["_rowpos"].map(lambda rp: dense_rank.get(int(rp), np.nan))
        out["rank_bm25"] = np.nan

        if anchor_date is not None:
            doc_day, age = _compute_time_arrays(df, union, anchor_date, date_col)
            out["doc_day"] = doc_day.dt.tz_localize(None)
            out["age_days"] = age
            if rank_time is not None:
                out["rank_time"] = rank_time

        if channel_w_col in out.columns:
            if w_channel is None:
                w_channel = 0.10 * float(np.std(out["score_rrf"].to_numpy(dtype=np.float32)) or 1.0)
            out["score_rrf"] = out["score_rrf"] + float(w_channel) * out[channel_w_col].astype(np.float32)
            out = out.sort_values("score_rrf", ascending=False)

        return out.head(k).reset_index(drop=True)

    bm_scores = bm25.get_scores(tokenize_fn(query)).astype(np.float32)
    if allowed_np is not None:
        bm_scores[~allowed_np] = -np.inf
    b_idx = _topk_indices_from_scores(bm_scores, topN_each)
    bm_rank = {int(rowpos): r for r, rowpos in enumerate(b_idx, start=1)}

    union = np.array(sorted(set(dense_rank) | set(bm_rank)), dtype=int)
    if len(union) == 0:
        return df.iloc[[]].copy().reset_index(drop=True)

    rrf = np.zeros(len(union), dtype=np.float32)
    for j, rowpos in enumerate(union):
        if rowpos in dense_rank:
            rrf[j] += w_dense / (k_rrf + dense_rank[rowpos])
        if rowpos in bm_rank:
            rrf[j] += w_bm25 / (k_rrf + bm_rank[rowpos])

    rank_time = None
    if anchor_date is not None and w_time and len(union) > 0:
        _, age = _compute_time_arrays(df, union, anchor_date, date_col)
        rank_time = _time_rank_from_age(age)
        rrf = rrf + (w_time / (k_rrf + rank_time.astype(np.float32)))

    order = np.argsort(-rrf)
    union = union[order]
    rrf = rrf[order]
    if rank_time is not None:
        rank_time = rank_time[order]

    out = df.iloc[union].copy()
    out["_rowpos"] = union
    out["score_rrf"] = rrf
    out["rank_dense"] = out["_rowpos"].map(lambda rp: dense_rank.get(int(rp), np.nan))
    out["rank_bm25"] = out["_rowpos"].map(lambda rp: bm_rank.get(int(rp), np.nan))

    if anchor_date is not None:
        doc_day, age = _compute_time_arrays(df, union, anchor_date, date_col)
        out["doc_day"] = doc_day.dt.tz_localize(None)

        out["age_days"] = age
        if rank_time is not None:
            out["rank_time"] = rank_time

    if channel_w_col in out.columns:
        if w_channel is None:
            w_channel = 0.10 * float(np.std(out["score_rrf"].to_numpy(dtype=np.float32)) or 1.0)
        out["score_rrf"] = out["score_rrf"] + float(w_channel) * out[channel_w_col].astype(np.float32)
        out = out.sort_values("score_rrf", ascending=False)

    return out.head(k).reset_index(drop=True)

In [18]:
df["date_day"].min(), df["date_day"].max()

(Timestamp('2023-09-23 00:00:00+0000', tz='UTC'),
 Timestamp('2025-09-08 00:00:00+0000', tz='UTC'))

In [19]:
# ! pip install transformers torch sentence-transformers accelerate vllm

In [20]:
from transformers import AutoTokenizer, AutoConfig
from vllm import LLM
import torch

MODEL = "Qwen/Qwen2.5-32B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)

cfg = AutoConfig.from_pretrained(MODEL, trust_remote_code=True)
native_ctx = getattr(cfg, "max_position_embeddings", None)
print("GPU:", torch.cuda.get_device_name(0))
print("VRAM(GB):", round(torch.cuda.get_device_properties(0).total_memory / (1024**3), 1))
print("Model max_position_embeddings:", native_ctx)

MAX_MODEL_LEN = 19200

model = LLM(
    model=MODEL,
    dtype="bfloat16",
    max_model_len=MAX_MODEL_LEN,
    gpu_memory_utilization=0.88
)


GPU: NVIDIA A100-SXM4-80GB
VRAM(GB): 79.3
Model max_position_embeddings: 32768
INFO 01-10 19:05:14 [utils.py:253] non-default args: {'dtype': 'bfloat16', 'max_model_len': 19200, 'gpu_memory_utilization': 0.88, 'disable_log_stats': True, 'model': 'Qwen/Qwen2.5-32B-Instruct'}
INFO 01-10 19:05:15 [model.py:514] Resolved architecture: Qwen2ForCausalLM
INFO 01-10 19:05:15 [model.py:1661] Using max model len 19200
INFO 01-10 19:05:15 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 01-10 19:05:17 [system_utils.py:136] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
(EngineCore_DP0 pid=3897) INFO 01-10 19:05:24 [core.py:93] Initializing a V1 LLM engine (v0.13.0) with config: model='Qwen/Qwen2.5-32B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-32B-In

(EngineCore_DP0 pid=3897) /home/mlcore/conda/lib/python3.10/site-packages/tvm_ffi/_optional_torch_c_dlpack.py:174: UserWarning: Failed to JIT torch c dlpack extension, EnvTensorAllocator will not be enabled.
(EngineCore_DP0 pid=3897) We recommend installing via `pip install torch-c-dlpack-ext`
(EngineCore_DP0 pid=3897)   warnings.warn(


(EngineCore_DP0 pid=3897) INFO 01-10 19:05:29 [cuda.py:351] Using FLASH_ATTN attention backend out of potential backends: ('FLASH_ATTN', 'FLASHINFER', 'TRITON_ATTN', 'FLEX_ATTENTION')


Loading safetensors checkpoint shards:   0% Completed | 0/17 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:   6% Completed | 1/17 [00:00<00:09,  1.67it/s]
Loading safetensors checkpoint shards:  12% Completed | 2/17 [00:01<00:09,  1.52it/s]
Loading safetensors checkpoint shards:  18% Completed | 3/17 [00:01<00:09,  1.48it/s]
Loading safetensors checkpoint shards:  24% Completed | 4/17 [00:02<00:08,  1.47it/s]
Loading safetensors checkpoint shards:  29% Completed | 5/17 [00:03<00:08,  1.48it/s]
Loading safetensors checkpoint shards:  35% Completed | 6/17 [00:04<00:07,  1.44it/s]
Loading safetensors checkpoint shards:  41% Completed | 7/17 [00:04<00:07,  1.39it/s]
Loading safetensors checkpoint shards:  47% Completed | 8/17 [00:05<00:06,  1.50it/s]
Loading safetensors checkpoint shards:  53% Completed | 9/17 [00:06<00:05,  1.50it/s]
Loading safetensors checkpoint shards:  59% Completed | 10/17 [00:06<00:04,  1.47it/s]
Loading safetensors checkpoint shards:  65% Completed | 11/17

(EngineCore_DP0 pid=3897) INFO 01-10 19:05:42 [default_loader.py:308] Loading weights took 11.72 seconds
(EngineCore_DP0 pid=3897) INFO 01-10 19:05:43 [gpu_model_runner.py:3659] Model loading took 61.0375 GiB memory and 16.508222 seconds
(EngineCore_DP0 pid=3897) INFO 01-10 19:05:55 [backends.py:643] Using cache directory: /home/mlcore/.cache/vllm/torch_compile_cache/acdcfb9698/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=3897) INFO 01-10 19:05:55 [backends.py:703] Dynamo bytecode transform time: 11.91 s
(EngineCore_DP0 pid=3897) INFO 01-10 19:06:15 [backends.py:226] Directly load the compiled graph(s) for compile range (1, 8192) from the cache, took 13.748 s
(EngineCore_DP0 pid=3897) INFO 01-10 19:06:15 [monitor.py:34] torch.compile takes 25.66 s in total
(EngineCore_DP0 pid=3897) INFO 01-10 19:06:16 [gpu_worker.py:375] Available KV cache memory: 7.22 GiB
(EngineCore_DP0 pid=3897) INFO 01-10 19:06:17 [kv_cache_utils.py:1291] GPU KV cache size: 29,568 tokens
(EngineCo

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:06<00:00,  7.38it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:04<00:00,  8.73it/s]


(EngineCore_DP0 pid=3897) INFO 01-10 19:06:28 [gpu_model_runner.py:4587] Graph capturing finished in 12 secs, took 4.11 GiB
(EngineCore_DP0 pid=3897) INFO 01-10 19:06:28 [core.py:259] init engine (profile, create kv cache, warmup model) took 45.70 seconds
INFO 01-10 19:06:30 [llm.py:360] Supported tasks: ['generate']


In [21]:
SYSTEM_PROMPT = """Ты — очень внимательный новостной аналитик. Ты пишешь аккуратный дайджест по новостям в стиле корреспондента.

Вход: запрос, актуальная дата (YYYY-MM-DD) и документы вида:
[id] date=YYYY-MM-DD channel(s)=<канал1; канал2; ...>
<текст>
Все документы датированы НЕ ПОЗЖЕ актуальной даты.

ОБЩИЕ ПРАВИЛА:
1) Пиши на русском. Используй ТОЛЬКО информацию из документов.
2) Не упоминай номера документов ([id]).
3) Не добавляй дат, которых нет в документах. Не используй “сегодня/вчера/недавно” — только YYYY-MM-DD.
4) Любые числа/точные значения/пороги/цитаты/формулировки — только если они есть в документах.
5) Каналы указывай ТОЛЬКО в дайджесте (в таймлайне каналов не будет).
6) Если рядом с актуальной датой мало данных — честно укажи последнюю дату в подборке и разрыв, без домыслов “что сейчас”.
7) Нельзя вставлять “пустые” даты и строки вида “нет данных”.

КАК ПИСАТЬ ДАЙДЖЕСТ:
- Начни с 2–4 самых свежих УНИКАЛЬНЫХ дат в подборке (это “последнее время”).
- Для каждого ключевого факта укажи источник:
  “Канал(ы) (YYYY-MM-DD): …”
- Не смешивай разные типы сигналов как одно и то же: явно различай
  “официальный курс / внебиржевой курс / расчетный курс / индекс доллара / прогноз”.
- Если встречаются повторы — объединяй.
- Пиши официально и нейтрально, как в новостном дайджесте (без эмоциональных оценок и без домыслов).
- Если в запросе есть структура вопроса, то ответь на него!

СТРУКТУРА ОТВЕТА (3 блока):

### 1) Запрос и актуальная дата
* Запрос: ...
* Актуальная дата: ...

### 2) Дайджест
Связный текст (8–20 предложений, не более!), как корреспондент:
- “Последнее время”: несколько ключевых фактов по самым свежим датам + кто сообщил.
- Затем коротко “раньше/предыстория”: несколько ключевых фактов по более ранним датам + кто сообщил.
- 1–2 предложения: выводы, исходя из актуальности новостей. Можно ли дать актуальную оценку или нет. Насколько тема в целом актуальная, свежая и вирусная.
- И ответ на вопрос, если запрос представлен в формате вопроса, кратко, лаконично на основе дайджеста!

### 3) Таймлайн (полный, 1 дата = 1 строка = 1 факт, без каналов)
Полный список ВСЕХ УНИКАЛЬНЫХ дат из документов, в порядке от самой старой к самой новой (ascending).
Одна строка = одна (1) дата.
Одна строка = один (1) самый релевантный запросу факт/изменение, который явно есть в документах этой даты.

Формат строки:
* YYYY-MM-DD — что сообщили / что изменилось (1 факт)

ПРАВИЛА ТАЙМЛАЙНА (критично):
- В таймлайне НЕ УКАЗЫВАЙ каналы вообще.
- Не добавляй дат, которых нет в документах!
- Не переноси факты между датами “по смыслу”: в строке даты могут быть только факты из документов этой даты!
- Никаких причин и выводов — только “что сообщили”!
- Если на одну дату приходится несколько фактов, выбери один самый важный для запроса (остальное оставь в дайджесте)!
! ЕСЛИ ДАТЫ НЕТ ВНУТРИ КОНТЕКСТА — ЕЁ НЕЛЬЗЯ УКАЗЫВАТЬ. ЕСЛИ ФАКТА НЕТ НА ЭТУ ДАТУ В ДОКУМЕНТАХ — ЕГО НЕЛЬЗЯ ПИСАТЬ. !
"""

import numpy as np
import pandas as pd

def build_rag_context(
    query: str,
    cand: pd.DataFrame,
    anchor_date: str,
    k_docs: int = 30,
    snip_chars: int = 850,
    hot_window_days: int = 30,
    hot_ratio: float = 0.8,
) -> str:
    if cand is None or len(cand) == 0:
        return (
            f"АКТУАЛЬНАЯ ДАТА ОБЗОРА: {anchor_date}\n"
            f"ВОПРОС/ЗАПРОС:\n{query}\n\n"
            f"ИСТОЧНИКИ:\n(нет документов)\n"
        )

    c = cand.copy()

    date_col = "date_day" if "date_day" in c.columns else "date"
    score_col = "score_temporal" if "score_temporal" in c.columns else "score_rrf"

    if "age_days" not in c.columns:
        ad = pd.to_datetime(anchor_date, utc=True).normalize()
        dts = pd.to_datetime(c[date_col], errors="coerce", utc=True).dt.normalize()
        c["age_days"] = (ad - dts).dt.days.astype("float32")

    age = c["age_days"].to_numpy(dtype=np.float32)
    hot_mask = (age >= 0) & (age <= float(hot_window_days))

    c = c.sort_values(score_col, ascending=False)

    n_hot = int(round(k_docs * float(hot_ratio)))
    n_hot = max(0, min(n_hot, k_docs))

    hot_part = c[hot_mask].head(n_hot)
    rest_part = c[~hot_mask].head(k_docs - len(hot_part))
    picked = pd.concat([hot_part, rest_part], axis=0)

    dd = pd.to_datetime(picked[date_col], errors="coerce", utc=True).dt.normalize()
    picked = picked.assign(_doc_day=dd).sort_values(["_doc_day", score_col], ascending=[False, False]).head(k_docs)

    blocks = []
    for i, row in enumerate(picked.itertuples(index=False), start=1):
        date_day = getattr(row, "date_day", getattr(row, "date", ""))
        if isinstance(date_day, pd.Timestamp):
            date_day = date_day.strftime("%Y-%m-%d")
        date_day = str(date_day)[:10]

        channel = getattr(row, "channel_name")
        text = getattr(row, "message", "")

        blocks.append(f"[{i}] date={date_day} channel(s)={channel}\n document=" + snippet(str(text), snip_chars))

    return (
        f"АКТУАЛЬНАЯ ДАТА ОБЗОРА: {anchor_date}\n"
        f"ВОПРОС/ЗАПРОС:\n{query}\n\n"
        f"ИСТОЧНИКИ:\n" + "\n\n".join(blocks)
    )


In [22]:
import re, json
import pandas as pd
from vllm import SamplingParams

JUDGE_SYSTEM = """Ты — строгий эксперт по информационному поиску по новостям (в т.ч. экономическим).

Твоя задача: оценить релевантность кандидатной новости запросу. Запрос может быть:
- коротким топиком (например "курс рубля к доллару"),
- или текстом другой новости (тогда запрос описывает конкретный инфоповод).

Используй ТОЛЬКО текст кандидатного документа. Ничего не додумывай.

Шкала релевантности:
2 — документ явно про то же самое: отвечает топику ИЛИ описывает тот же инфоповод/факт/событие, что и запрос.
1 — документ связан по теме/контексту, но это немного другой инфоповод, или про то же, но без прямого соответствия.
0 — нерелевантно совсем.

Правило строгости:
ставь 2 только если связь очевидна по тексту документа; если информации недостаточно — ставь 0 или 1.

Верни строго валидный JSON и ничего больше:
{"relevance": 0|1|2}
"""


def _parse_relevance(text: str) -> int:
    text = text.strip()
    m = re.search(r"\{.*\}", text, flags=re.DOTALL)
    if m:
        blob = m.group(0)
        try:
            obj = json.loads(blob)
            val = int(obj.get("relevance", 0))
            return val if val in (0, 1, 2) else 0
        except Exception:
            pass
    m2 = re.search(r"relevance\"\s*:\s*([012])", text)
    if m2:
        return int(m2.group(1))
    return 0

def judge_filter_candidates(
    cand: pd.DataFrame,
    query: str,
    judge_llm,
    judge_tokenizer,
    *,
    keep_threshold: int = 1,     
    doc_max_chars: int = 1200,
    batch_size: int = 32,
    max_out_tokens: int = 40,
) -> pd.DataFrame:
    if cand is None or len(cand) == 0:
        return cand

    text_col = "message"
    channel_col = "channel_name"
    date_col = "date_day"

    prompts = []
    for _, row in cand.iterrows():
        doc = str(row[text_col])[:doc_max_chars]
        ch = str(row[channel_col]) if channel_col else ""
        dt = str(row[date_col]) if date_col else ""

        user_msg = (
            f"ЗАПРОС:\n{query}\n\n"
            f"КАНДИДАТ:\n"
            f"channel={ch}\n"
            f"date={dt}\n"
            f"text:\n{doc}\n"
        )

        messages = [
            {"role": "system", "content": JUDGE_SYSTEM},
            {"role": "user", "content": user_msg},
        ]
        prompt = judge_tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        prompts.append(prompt)

    sampling = SamplingParams(
        temperature=0.0,
        top_p=1.0,
        max_tokens=max_out_tokens,
    )

    relevances = []
    for i in range(0, len(prompts), batch_size):
        batch_prompts = prompts[i:i+batch_size]
        outs = judge_llm.generate(batch_prompts, sampling)
        for o in outs:
            txt = o.outputs[0].text
            relevances.append(_parse_relevance(txt))

    out_df = cand.copy()
    out_df["judge_relevance"] = relevances

    filtered = out_df[out_df["judge_relevance"] >= keep_threshold].copy()
    filtered.reset_index(drop=True, inplace=True)
    return filtered


In [23]:
# import torch

# @torch.inference_mode()
# def rag_summarize(sum_model, sum_tokenizer, query: str, cand: pd.DataFrame, anchor_date, 
#                   k_docs: int = 25, snip_chars: int = 900, max_new_tokens: int = 2000) -> str:
    
#     user = build_rag_context(query, cand, anchor_date=anchor_date, k_docs=k_docs, snip_chars=snip_chars)
#     print("built context...")
    
#     messages = [
#         {"role": "system", "content": SYSTEM_PROMPT},
#         {"role": "user", "content": user},
#     ]
#     prompt = sum_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
#     enc = sum_tokenizer(prompt, return_tensors="pt", truncation=True).to(sum_model.device)

#     out_ids = sum_model.generate(
#         **enc,
#         max_new_tokens=max_new_tokens,
#         do_sample=False,
#         eos_token_id=sum_tokenizer.eos_token_id,
#         pad_token_id=sum_tokenizer.eos_token_id,
#     )
#     prompt_len = int(enc["attention_mask"][0].sum().item())
    
#     return sum_tokenizer.decode(out_ids[0][prompt_len:], skip_special_tokens=True).strip(), user


In [24]:
from vllm import SamplingParams

def rag_summarize(
    sum_model,
    sum_tokenizer,
    query: str,
    cand: pd.DataFrame,
    anchor_date,
    k_docs: int = 25,
    snip_chars: int = 900,
    max_new_tokens: int = 2000,
    hot_window_days: int = 30,
    hot_ratio: float = 0.8,
):
    user = build_rag_context(
        query=query,
        cand=cand,
        anchor_date=str(anchor_date),
        k_docs=k_docs,
        snip_chars=snip_chars,
        hot_window_days=hot_window_days,
        hot_ratio=hot_ratio,
    )
    print("built context...")

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user},
    ]

    prompt = sum_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    sampling = SamplingParams(
        temperature=0.0,
        top_p=1.0,
        max_tokens=max_new_tokens,
    )

    result = sum_model.generate([prompt], sampling)[0]
    text = result.outputs[0].text.strip()
    return text, user


In [25]:
def run_rag_hybrid(
    df: pd.DataFrame,
    index,
    encoder,
    bm25,
    tokenize_fn,
    query: str,
    k_retrieve: int = 50,
    topN_each: int = 500,
    k_docs: int = 25,
    snip_chars: int = 1500,
    max_new_tokens: int = 2000,
    anchor_date: str = "2025-09-04",
    max_window_days: int | None = 365,
    w_time: float = 0.5,
    w_channel: float | None = None,
    hot_window_days: int = 30,
    hot_ratio: float = 0.8,
    sum_model=None,
    sum_tokenizer=None,
    judge_llm=None,
    judge_tokenizer=None,
    judge_keep_threshold: int = 1,
    judge_batch_size: int = 32,
):
    cand = hybrid_retrieve_rrf(
        df=df,
        index=index,
        encoder=encoder,
        bm25=bm25,
        tokenize_fn=tokenize_fn,
        query=query,
        k=k_retrieve,
        topN_each=topN_each,
        k_rrf=60,
        w_dense=1.0,
        w_bm25=1.0,
        anchor_date=anchor_date,
        max_window_days=max_window_days,
        w_time=w_time,
        w_channel=w_channel,
    )
    print("retrieval done...")
    cand_before = cand

    cand_clusters = None
    members = None
    if cand is not None and len(cand) > 0 and encoder is not None:
        text_col = "message"
        cand, cand_clusters, members = dedup_cluster_candidates_time(
            cand=cand,
            encoder=encoder,
            text_col=text_col,
            score_col="score_rrf",
            sim_threshold=0.95,
            knn=30,
            keep_per_cluster=1,
            mask_numbers=False,
            max_day_diff=1,
            overwrite_channel=True,
        )
    print("clustering done...")

    cand_after_dedup = cand
    if (
        judge_llm is not None
        and judge_tokenizer is not None
        and cand is not None
        and len(cand) > 0
    ):
        cand = judge_filter_candidates(
            cand=cand,
            query=query,
            judge_llm=judge_llm,
            judge_tokenizer=judge_tokenizer,
            keep_threshold=judge_keep_threshold,
            doc_max_chars=snip_chars,
            batch_size=judge_batch_size,
        )
    print("filtering done...")

    if sum_model is None or sum_tokenizer is None:
        ctx = build_rag_context(
            query,
            cand,
            anchor_date=anchor_date,
            k_docs=min(k_docs, len(cand)) if cand is not None else 0,
            snip_chars=snip_chars,
            hot_window_days=hot_window_days,
            hot_ratio=hot_ratio,
        )
        return {
            "context": ctx,
            "candidates": cand_before,
            "candidates_dedup": cand_after_dedup,
            "candidates_filtered": cand,
            "members": members,
            "clusters": cand_clusters,
            "summary": "No LLM",
        }

    summary, ctx = rag_summarize(
        sum_model,
        sum_tokenizer,
        query,
        cand,
        k_docs=min(k_docs, len(cand)) if cand is not None else 0,
        snip_chars=snip_chars,
        max_new_tokens=max_new_tokens,
        anchor_date=anchor_date,
        hot_window_days=hot_window_days,
        hot_ratio=hot_ratio,
    )
    print("summary done...")

    return {
        "context": ctx,
        "summary": summary,
        "candidates": cand_before,
        "candidates_dedup": cand_after_dedup,
        "candidates_filtered": cand,
        "members": members,
        "clusters": cand_clusters,
    }


In [26]:
# import gc, torch

# try:
#     del model
# except Exception:
#     pass

# gc.collect()
# torch.cuda.empty_cache()
# torch.cuda.synchronize()


In [27]:
q = "Почему кофе дорожает?"

out = run_rag_hybrid(
    df=df,
    index=index,
    encoder=encoder,
    bm25=bm25,
    tokenize_fn=tokenize_ru,
    query=q,
    k_retrieve=150,
    topN_each=2000,
    k_docs=50,
    snip_chars=1000,
    max_new_tokens=5000,
    anchor_date="2025-09-04",
    max_window_days=365,
    w_time=0.5,
    w_channel=None,
    hot_window_days=30,
    hot_ratio=0.7,
    sum_model=model,
    sum_tokenizer=tokenizer,
    judge_llm=model,
    judge_tokenizer=tokenizer,
    judge_keep_threshold=1,
    judge_batch_size=32)


retrieval done...
clustering done...


Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...

Processed prompts:   0%|          | 0/6 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

filtering done...
built context...


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

summary done...


In [28]:
from IPython.display import display, Markdown, HTML
import re

def show_summary(summary: str):
    if summary is None:
        display(HTML("<b>summary is None</b>"))
        return

    s = str(summary)
    s = s.replace("\\n", "\n") 
    s = re.sub(r"\n{3,}", "\n\n", s).strip()

    display(Markdown(s))

show_summary(out["summary"])


### 1) Запрос и актуальная дата
* Запрос: Почему кофе дорожает?
* Актуальная дата: 2025-09-04

### 2) Дайджест
Последнее время, с 2025-08-30, мировые цены на кофе сорта робуста и арабика подорожали на 1,5 и 29% соответственно, что связано с проблемами с урожаем в Бразилии и Вьетнаме из-за неблагоприятных погодных условий. В рамках проекта по борьбе с производством кокаина в Колумбии, фермеров убеждают выращивать кофе, что может в долгосрочной перспективе повлиять на рынок. В 2025-08-24, Coca-Cola рассматривает возможность продажи сети кофеен Costa Coffee из-за роста цен на кофейные зерна и конкуренции с другими кофейнями.

Ранее, в 2025-06-26, эксперт Кирилл Селезнев предсказывал снижение мировых цен на кофе до конца года на 10-20%, но погодные аномалии могут помешать этому. В 2025-04-19, колумбийский эксперт Себастьян Сулуага предупреждал, что цены на кофе могут стабилизироваться, но каждый вид кофе будет стоить по-своему на бирже.

Тема подорожания кофе остается актуальной и вирусной, особенно в контексте неблагоприятных погодных условий и проблем с урожаем. В России кофе может подорожать на 20-40% из-за засухи в Бразилии и Вьетнаме, что подтверждается различными экспертами.

### 3) Таймлайн
* 2025-09-01 — Фермеров в Колумбии убеждают выращивать кофе вместо коки
* 2025-08-30 — Мировые цены на кофе сорта робуста и арабика подорожали на 1,5 и 29% соответственно
* 2025-08-24 — Coca-Cola рассматривает возможность продажи сети кофеен Costa Coffee
* 2025-08-19 — Индекс кофе с бутербродом подешевел, но кофе и хлеб подорожали
* 2025-08-01 — Ожидается новое подорожание кофе на 20-40% из-за проблем с урожаем в Бразилии и Вьетнаме
* 2025-07-18 — Розничные продажи растворимого кофе сократились, но интерес к кофе в целом вырос
* 2025-07-11 — Повышение импортных пошлин США на товары из Бразилии может остановить поставки бразильского кофе
* 2025-06-26 — Эксперт предсказывает снижение мировых цен на кофе до конца года на 10-20%
* 2025-06-16 — Кофе и какао могут подорожать из-за торговых войн
* 2025-06-07 — Эфиопия готовится собрать рекордный урожай кофе
* 2025-05-14 — Торги фьючерсами на кофе на срочном рынке Мосбиржи начнутся с 20 мая
* 2025-04-30 — В России растет интерес к альтернативным горячим напиткам из-за дорожающего кофе
* 2025-04-19 — Эксперт предупреждает о возможной стабилизации цен на кофе
* 2025-04-07 — Кофе в России может подорожать на 50% из-за засухи в Бразилии и роста логистических расходов
* 2025-03-25 — Большинство москвичей остро переживают резкое подорожание кофе
* 2025-03-24 — Кофе может подорожать на 40% из-за засухи в Бразилии и роста логистических издержек
* 2025-03-17 — Чай подорожает в 2025 году из-за роста стоимости сырья и логистики
* 2025-03-07 — Мировая торговля кофе сократилась из-за резкого роста цен
* 2025-03-04 — Кофе может подорожать на 30-40% из-за усложнения логистики и изменений климата
* 2025-02-09 — Растворимый кофе в магазинах подорожал до 1,5 тысячи рублей
* 2025-01-30 — Поставщики собираются увеличить цены на кофе на 20% с марта
* 2025-01-29 — Биржевые цены на кофе сорта арабика достигли рекордного уровня
* 2025-01-27 — Биржевые цены на арабику достигли нового рекорда
* 2025-01-10 — Россияне переходят на домашнее приготовление кофе из-за высоких цен
* 2024-12-23 — Кофе в России крайне сильно подорожает в 2025 году из-за урожая арабики
* 2024-12-13 — Цены на кофе продолжают расти и могут потребовать времени на снижение
* 2024-12-10 — Мосбиржа запустит сделки с фьючерсами на кофе и апельсиновый сок
* 2024-12-04 — Импортные товары, включая кофе, подорожают на 10-20% из-за падения рубля
* 2024-12-02 — Поставщики считают, что следующий год станет «большим испытанием» для любителей кофе
* 2024-12-01 — Кофе в России станет дороже на 25% из-за засухи в Бразилии и тайфунов во Вьетнаме
* 2024-11-27 — Стоимость кофе сорта арабика достигла максимума с 1972 года
* 2024-11-25 — Биржевые цены на кофе сорта арабика находятся вблизи новых рекордов
* 2024-10-31 — Продажи кофе на вынос в России выросли на четверть в денежном выражении
* 2024-10-24 — Продажи кофе на вынос выросли на 25% за год
* 2024-10-01 — Цены на зерновой кофе удвоились и продолжат расти из-за засухи в Бразилии и Вьетнаме
* 2024-09-26 — Кофе в России резко подорожает до 30% к концу года из-за засухи в Бразилии и Вьетнаме
* 2024-09-16 — Цены на кофе взлетели до максимума за 13 лет из-за снижения производства кофе в Бразилии

In [29]:
print("context chars:", len(out["context"]))
len(SYSTEM_PROMPT)

context chars: 19663


2900

In [30]:
show_summary(out["context"])

АКТУАЛЬНАЯ ДАТА ОБЗОРА: 2025-09-04
ВОПРОС/ЗАПРОС:
Почему кофе дорожает?

ИСТОЧНИКИ:
[1] date=2025-09-01 channel(s)=Forbes Russia
 document=Сын миллиардера Баффета Говард финансирует проект по борьбе с производством кокаина в Колумбии стоимостью $170 млн. В рамках проекта фермеров в регионах, где процветает наркоторговля, убеждают выкорчевать свои кусты коки. И финансово помогают им выращивать вместо коки легальные культуры, в том числе кофе, персиковую пальму и какао

[2] date=2025-08-30 channel(s)=Раньше всех. Ну почти
 document=По итогам августа пиломатериалы и медь больше всего потеряли в цене на мировых рынках. При этом самыми подорожавшими товарами в этом месяце были сорта кофе робуста и арабика, сообщает РИА Новости со ссылкой на данные биржи ICE Futures, CME Group и Шанхайской биржи металлов. Стоимость кофе сорта робуста за месяц увеличилась в 1,5 раза, арабики — на 29%. Самым подорожавшим металлом стал неодим (используется для производства магнитов) — цена выросла на 20%. Цены на литий увеличились на 9,2%, на молибден (нужен для сплавов и катализаторов) — на 7,7%.

[3] date=2025-08-26 channel(s)=Банки, деньги, два офшора
 document=Шоколад для россиян станет таким же дорогим и дефицитным продуктом, как чёрная икра, заявила вице-президент Российской гильдии пекарей и кондитеров Ирина Эльдарханова. Она призвала не ждать возвращения прежних цен, отметив, что шоколад никогда не дешевеет из-за нехватки качественного сырья. Какао-бобы выросли в цене больше чем в два раза, другое сырье — на 25%, из-за чего люди, которые хотят есть качественный шоколад, в любом случае столкнутся с ростом цен. @bankrollo

[4] date=2025-08-24 channel(s)=Forbes Russia
 document=Coca-Cola изучает возможность продажи сети кофеен Costa Coffee, пишет Financial Times со ссылкой на источники. Причинами названы рост цен на кофейные зерна и конкуренция с другими кофейнями, которые негативно сказались на показателях компании. Coca-Cola купила самую большую в Великобритании сеть кофеен Costa Coffee в 2018 году за £3,9 млрд

[5] date=2025-08-19 channel(s)=Банки, деньги, два офшора
 document=Бутербродекс подешевел на 2,1 пункта впервые за лето. Индекс кофе с бутербродом, отражающий образ популярного у россиян завтрака, теперь составляет 147,9 пункта. Согласно данным, в корзине подешевело всё, кроме кофе и хлеба. @bankrollo

[6] date=2025-08-01 channel(s)=Блумберг
 document=В России ожидается новое подорожание кофе — на 20–40%. Причина — проблемы с урожаем в Бразилии и Вьетнаме из-за жары и дождей. На фоне дефицита мировые цены растут, а в российских магазинах упаковка 150 г уже стоит в среднем 379₽ — на 30% дороже, чем год назад. 🔵 Bloomberg

[7] date=2025-07-18 channel(s)=Forbes Russia
 document=Розничные продажи растворимого кофе за последние 12 месяцев сократились почти на 5%, подсчитали в аналитической компании «Нильсен». Одновременно зафиксирован общий рост интереса к кофе: в натуральном выражении его совокупные розничные продажи за 12 месяцев увеличились на 1,3%. В кофейном бренде Poetti указали на развитие кофейной культуры в России: потребители делают более осознанный выбор, ценят качество и происхождение зерен

[8] date=2025-07-11 channel(s)=Forbes Russia
 document=Повышение до 50% импортных пошлин США на товары из Бразилии может остановить поставки бразильского кофе в эту страну. Треть потребляемого в США кофе поступает из Бразилии, крупнейшего в мире его производителя, а Соединенные Штаты — крупнейший потребитель, пишет Reuters. Проблемными продуктами для ввоза в США из Бразилии также станут апельсиновый сок и биоэтанол (биотопливо на основе сахарного тростника или кукурузы)

[9] date=2025-07-10 channel(s)=Банки, деньги, два офшора
 document=Кофе подорожает на 40% в этом году — прогноз экспертов. Цены вырастут из-за экстремальной погоды в странах-производителях. В Бразилии из-за засухи, во Вьетнаме — проливных дождей. Вскоре одна чашка кофе будет стоить минимум 500 рублей. @bankrollo

[10] date=2025-06-26 channel(s)=Риа Новости
 document=Мировые цены на кофе до конца года могут снизиться на 10-20%, рассказал РИА Новости эксперт по фондовому рынку "Гарда Капитал" Кирилл Селезнев. По данным биржи ICE, цены на арабику в мире во вторник упали на 4,7%, достигнув самого низкого уровня с декабря 2024-го – $6856 за тонну. Если погодные аномалии не испортят оптимистичных планов по урожаю, цены могут постепенно остывать и далее... Арабика может опуститься в цене ниже $6 тысяч за тонну, робуста может подешеветь до $3 тысяч и ниже. 🟦 Подписаться на РИА Новости / Все наши каналы

[11] date=2025-06-16 channel(s)=Риа Новости
 document=Кофе и какао в мире могут подорожать из-за торговых войн, рассказала РИА Новости в преддверии ПМЭФ гендиректор рейтингового агентства "Эксперт РА" Марина Чекурова. "Она может возникать в отдельных секторах - например, рынок кофе или какао должен беспокоить нас как потребителей, но в целом должна быть под контролем", - ответила она на вопрос, что будет с инфляцией в условиях замедления мировых экономик на фоне торговых войн. Стоимость кофе сорта арабика в прошлом году начала обновлять рекорды: в начале февраля биржевые цены превысили $8,5 тысячи за тонну. А какао-бобы и вовсе стали самым подорожавшим товаром за прошлый год, увеличившись в цене на 181%.

[12] date=2025-06-07 channel(s)=Раньше всех. Ну почти
 document=Эфиопия готовится собрать рекордный урожай кофе в 11,56 млн мешков, или 694 тыс. тонн, в наступившем сельскохозяйственном сезоне 2025/2026 гг, сообщил телеканал Afrique Media со ссылкой на данные Национального управления кофе и чая Эфиопии. Прогнозируемый результат на 9% выше предыдущего показателя в 638 тыс. тонн, отмечает телеканал.

[13] date=2025-05-14 channel(s)=Forbes Russia
 document=Торги фьючерсами на кофе на срочном рынке Мосбиржи начнутся с 20 мая. Базовым активом станет кофе сорта арабика, который торгуется на зарубежных рынках. На начальном этапе инвесторам будут доступны контракты с исполнением в июне и августе нынешнего года

[14] date=2025-05-14 channel(s)=Раньше всех. Ну почти
 document=Московская биржа с 20 мая запустит торги фьючерсами на кофе — РБК

[15] date=2025-04-30 channel(s)=Forbes Russia
 document=Кофе становится все дороже, и на этом фоне в России растет интерес к разного рода альтернативным горячим напиткам, таким, как цикорий. «Согласно различным экспертным мнениям, объем российского рынка цикория составляет около 2-3 млрд рублей в год, — подсчитал директор департамента управленческого консалтинга группы «Деловой профиль» Владимир Поклад. — Это относительно скромная цифра по сравнению с рынком кофе (более 100 млрд рублей), однако динамика роста сегмента выглядит впечатляюще». По словам экспертов, опрошенных Forbes, потребление цикория постепенно входит в моду на волне интереса к здоровому образу жизни, так что дело не только в экономии. Подробнее о том, почему в России растет потребление цикория — рассказываем на сайте 📸: Фото Getty Images

[16] date=2025-04-19 channel(s)=Риа Новости
 document=Колумбийский эксперт и бариста Себастьян Сулуага рассказал РИА Новости, что будет с беспрецедентно выросшими ценами на кофе в 2025 г: «Скорее всего, рост продолжится, но настанет момент, когда пузырь лопнет… Цена может стабилизироваться, но каждый отдельный вид будет по-своему стоить на бирже», - поделился прогнозом специалист, отметив, что удорожание кофе многие в отрасли связывают с действиями спекулянтов на рынке. В феврале биржевая стоимость кофе арабика поднималась до исторического максимума и превышала отметку в $9,5 тыс за тонну.

[17] date=2025-04-15 channel(s)=Банки, деньги, два офшора
 document=Кражи стаканчиков в кофейных аппаратах разоряют предпринимателей. Владелец кофе-автомата в одном из «Магнитов» показал, с чем он постоянно сталкивается — неизвестные похищают все бумажные стаканы. Это приводит к простоям и существенным убыткам, ведь кофе не во что наливать. @bankrollo

[18] date=2025-04-07 channel(s)=Банки, деньги, два офшора; Блумберг
 document=Кофе в России может подорожать на 50% уже в этом году, предупреждают аналитики Last Wish. Причины — засуха в Бразилии, рост логистических расходов, дефицит запасов и разбалансировка мирового спроса. Больше всего прибавит в цене арабика. 🔵 Bloomberg

[19] date=2025-03-25 channel(s)=Банки, деньги, два офшора
 document=Москвичи отказываются от кофе из-за космических цен. Большинство жителей столицы (70%) остро переживают резкое подорожание напитка. Почти треть даже готовы сокращать его потребление, показал опрос Шоколадницы. Только 1,9% граждан готовы покупать чашку капучино за 350 рублей и выше. Большинство считают приемлемой стоимость ниже 200 рублей, но на столичном рынке уже практически нет таких цен. @retailrus

[20] date=2025-03-24 channel(s)=Банки, деньги, два офшора
 document=Кофе может резко подорожать до 40% уже в этом году, предупреждают участники рынка. Сильнее всего подорожают сорта арабика и органический кофе. Это произойдёт из-за засухи в Бразилии и роста логистических издержек. @bankrollo

[21] date=2025-03-24 channel(s)=Блумберг
 document=Россиян предупредили о резком скачке цен на кофе. Арабика может подорожать до 40% в 2025 году, если всё пойдёт по худшему сценарию, заявили аналитики. При спокойной обстановке — стабильный рубль и без погодных катаклизмов — рост будет 5–15%, а вот засуха в Бразилии или прыжок доллара могут задрать цены на 20–40%. Робуста, наоборот, почти не подороджает. Сейчас арабика с робустой в магазинах стоит по-разному: 250 г молотой смеси — около 575 рублей, килограмм зёрен смеси — 1,6 тысячи, а чистая арабика на килограмм тянет на 4,2 тысячи. 🔵 Bloomberg

[22] date=2025-03-17 channel(s)=Банки, деньги, два офшора; Блумберг
 document=Чай подорожает в 2025 году. Элитные сорта, включая улун и пуэр, могут прибавить 25–40% в цене, а пакетированный чай — 10–15%. Основные факторы — рост стоимости сырья, упаковки, логистики, инфляция и увеличение спроса на фоне подорожания кофе. 🔵 Bloomberg

[23] date=2025-03-07 channel(s)=Forbes Russia
 document=Мировая торговля кофе практически остановилась, сообщает Reuters. По данным агентства, из-за резкого роста цен мировые торговцы и обжарщики кофе сократили закупки кофейных бобов до минимума. Причиной стало снижение объемов производства в регионах, где выращиваются кофейные зерна, включая Бразилию. В ассоциации «Росчайкофе» ранее предупреждали, что для любителей кофе в России 2025 год будет «большим испытанием»

[24] date=2025-03-07 channel(s)=Раньше всех. Ну почти
 document=Мировая торговля кофе практически остановилась из-за того, что продавцы напитка сократили закупки кофейных бобов до минимума из-за резкого роста цен на них, сообщило агентство Reuters. По его информации, участники ежегодного съезда американской Национальной ассоциации кофе, который прошел в штате Техас на прошлой неделе, заявили, что были шокированы ростов фьючерсов на сорт кофе арабика в 70% на бирже ICE. По словам генерального директора производителя растворимого кофе из Эквадора Elcafe Ренана Чуэйри, в этом году компания впервые не смогла продать весь объем товара, который предполагалось реализовать к марту.

[25] date=2025-03-04 channel(s)=Банки, деньги, два офшора
 document=Кофе рекордно подорожает на 30-40% уже к концу года, пишет Газета.ру. Среди ключевых причин роста цен эксперты называют усложнение логистики, санкции, рост цен на рабочую силу и топливо, а также изменение климата в странах-производителях. Средняя цена за килограмм кофе в 2024 году составила 1–2 тысячи рублей. @bankrollo

[26] date=2025-02-09 channel(s)=Банки, деньги, два офшора
 document=Растворимый кофе в магазинах начали продавать уже за 1,5 тысячи рублей. @bankrollo

[27] date=2025-01-30 channel(s)=Forbes Russia
 document=Крупнейшие поставщики собираются увеличить отпускные цены на кофе в среднем на 20% с марта, узнали «Ведомости». Источники в ретейле также ожидают и других повышений в текущем году. Поставщики объясняют это в том числе засухой и проблемами с сырьем. Это общемировой тренд, напоминают аналитики. За 2024 год биржевые цены на зерна робусты выросли на 62%, на арабику — почти на 70%

[28] date=2025-01-30 channel(s)=Блумберг
 document=Цены на кофе в России подскочат ещё на 20% уже весной. Крупнейшие поставщики уже уведомили торговые сети о повышении отпускных цен на зерна. Ритейлеры говорят, что это не единственное повышение, которое случится в течение 2025 года. На стоимости отражается в том числе неурожай в Бразилии и Вьетнаме 🔵 Bloomberg

[29] date=2025-01-29 channel(s)=Риа Новости
 document=Биржевые цены на кофе сорта арабика вновь бьют рекорды – уже поднялись впервые в истории выше $8 тыс за тонну. Стоимость арабики за 2024 выросла в 1,7 раза. Эксперты полагают, что в целом цены на кофе в этом году будут повышаться – виной тому риски для нового урожая из-за ожидающейся непогоды.

[30] date=2025-01-27 channel(s)=Риа Новости
 document=Биржевые цены на арабику достигли нового рекорда – более $7,8 тыс за тонну. За 2024 стоимость арабики выросла в 1,7 раза. Эксперты полагают, что цены на дорожавший в прошлом году кофе будут расти и в этом – новый урожай под угрозой из-за ожидающейся непогоды.

[31] date=2025-01-10 channel(s)=Блумберг
 document=Россияне всё чаще отказываются от кофе навынос из-за высоких цен и переходят на домашнее приготовление. Продажи кофемашин выросли на 30% за год, сообщили ретейлеры. Популярны устройства со встроенными капучинаторами стоимостью около 46 700 рублей. Цены на напитки в кофейнях доходят до 800 рублей за стакан, что связано с подорожанием зерна из-за неурожая в Бразилии и Вьетнаме. 🔵 Bloomberg

[32] date=2025-01-09 channel(s)=Экономика 
 document=Цены на кофе и шоколад в этом году взлетят на 25%, предупреждают эксперты. Рекорды на биржах уже привели к повышению цен. Летом кофейни подняли стоимость продукции минимум на 30 рублей, и рост продолжится в течение всего 2025 года. 🤑 The Экономист

[33] date=2024-12-23 channel(s)=Банки, деньги, два офшора
 document=Кофе в России крайне сильно подорожает в 2025 году. Всё из-за улетевшей в космос арабики, пояснили в МилФудс (бренд Poetti). На этом фоне некоторые российские производители уже закладывают изменения в свои отпускные цены несмотря на запасы. 2025 год станет «большим испытанием» для любителей кофе в РФ, потому что перерасчёт цен идёт постоянно. @bankrollo

[34] date=2024-12-13 channel(s)=Forbes Russia
 document=Цены на кофе продолжают расти и после достижения рекордных значений, на их снижение могут потребоваться годы, прогнозируют аналитики. На этой неделе фьючерсы на кофе сорта арабика поднимались до максимума почти за 50 лет. Цены на кофе растут на фоне опасений из-за урожая 2025 года в Бразилии, где после засухи осенью начались ливни

[35] date=2024-12-10 channel(s)=Банки, деньги, два офшора; Сигналы РЦБ; Экономика 
 document=Мосбиржа в 2025 году запустит сделки с фьючерсами на кофе и апельсиновый сок. В конце сентября биржа уже запустила фьючерс на какао. @bankrollo

[36] date=2024-12-10 channel(s)=Раньше всех. Ну почти
 document=Кофе стал одним из самых востребованных товаров года. Цены на него достигли рекордного уровня из-за возможного глобального дефицита, сообщило агентство Bloomberg. По его данным, фьючерсы на арабику в Нью-Йорке торгуются на уровне $3,44 за фунт. Цены превзошли предыдущий исторический максимум, установленный в 1977 году, отмечает агентство.

[37] date=2024-12-04 channel(s)=Банки, деньги, два офшора
 document=Чай, кофе, тропические фрукты и другие импортные товары подорожают на 10–20% в ближайшие месяц-полтора из-за падения рубля, пишет Forbes. Помимо роста цен на импортные продукты из-за ослабления рубля перед новогодними праздниками на 5–10% могут подорожать и продукты, которые производятся в России. @bankrollo

[38] date=2024-12-04 channel(s)=Экономика 
 document=Фрукты, чай и кофе из-за рубежа взлетят в цене на 10–20% уже в этом и следующем месяцах, сообщают эксперты. Эти товары сильнее всего реагируют на изменение курса валют. Также на 5–10% подорожают продукты, которые производятся в России. 🤑 The Экономист

[39] date=2024-12-02 channel(s)=Forbes Russia
 document=Поставщики кофе считают, что следующий год станет для российских любителей кофе «большим испытанием», пишет РБК. За 2024 год биржевые цены на робусту выросли более чем в два раза, а на арабику — на 80%. Участники рынка прогнозируют дальнейший рост розничной стоимости кофе, однако полагают, что россияне продолжат его приобретать, несмотря на подорожание

[40] date=2024-12-01 channel(s)=Банки, деньги, два офшора
 document=Кофе в России станет дороже ещё на 25% из-за засухи в Бразилии и тайфунов во Вьетнаме, предупредили участники рынка. @bankrollo

[41] date=2024-12-01 channel(s)=Блумберг; Экономика 
 document=В следующем году кофе в России может подорожать на 25%. Эксперты объясняют это снижением урожая у крупнейших производителей арабики и робусты — Бразилии и Вьетнама, что вызвано неблагоприятными погодными условиями. 🔵 Bloomberg

[42] date=2024-11-27 channel(s)=Блумберг; Раньше всех. Ну почти
 document=Стоимость кофе сорта арабика продолжает расти на фоне опасений за неурожай в Бразилии. Фьючерсы на арабику в среду подорожали более чем на 3% и достигли $3,22 за фунт на торгах в Нью-Йорке. По данным Trading Economics, это самый высокий показатель с 1972 года. 🔵 Bloomberg

[43] date=2024-11-25 channel(s)=Раньше всех. Ну почти
 document=Биржевые цены на кофе сорта арабика в понедельник находятся вблизи новых рекордов с 2011 года, свидетельствуют данные торгов. По состоянию на 14.21 мск стоимость мартовского фьючерса на арабику росла на 0,9%, до 3,0483 доллара за фунт, или примерно до 6 720 долларов за тонну.

[44] date=2024-10-31 channel(s)=Сигналы РЦБ
 document=☕ Продажи кофе на вынос в России за год увеличились на четверть в денежном выражении. Напиток становится все популярнее и у потребителей, и у предпринимателей. Активно развиваются сети кофе-пойнтов и вендинговых автоматов — Forbes

[45] date=2024-10-31 channel(s)=Банки, деньги, два офшора
 document=Капучино и латте резко подорожают из-за роста цен на зерно и молоко, предупреждают владельцы кофеен в России. С начала года цены на молоко и сливки подскочили на 30-35%. Стоимость кофейных зёрен арабики выросла на 40%, робусты — на 80%. Cofix уже повысил цены на 30 рублей, One Price Coffee — на 20. Даблби также будет вынуждено корректировать цены на кофе. @bankrollo

[46] date=2024-10-24 channel(s)=Сигналы РЦБ
 document=Продажи кофе на вынос выросли на 25% за год — Forbes Кофе-пойнты и вендинг становятся основными драйверами этого сегмента рынка.

[47] date=2024-10-24 channel(s)=Сигналы РЦБ
 document=☕ Количество покупок кофе с собой за 9мес2024 выросло на 14% г/г— Ведомости Популярность напитка растет у молодежи, которая не слишком чувствительна к цене.

[48] date=2024-10-01 channel(s)=Блумберг
 document=Цены на зерновой кофе удвоились и продолжат расти Причина в засухе и сокращении урожайности в Бразилии и Вьетнаме. Биржевые цены на зерно за последний год выросли почти вдвое, а за полгода — на 42%, достигнув $2,7 за фунт (0,45 кг), что является максимумом за последние 13 лет. Крупные кофейни сообщают о подорожании кофе минимум на 35%. 🔵 Bloomberg

[49] date=2024-09-26 channel(s)=Банки, деньги, два офшора
 document=Кофе в России резко подорожает до 30% к концу года, предупреждают участники рынка. Цены взлетят в магазинах и кофейнях из-за засухи в Бразилии и Вьетнаме. @bankrollo

[50] date=2024-09-16 channel(s)=Банки, деньги, два офшора
 document=Цены на кофе взлетели до максимума за 13 лет. Как пишет Bloomberg, в 2024 году стоимость арабики поднялась примерно на 40% из-за снижения производства кофе в Бразилии, которая является одним из крупнейших поставщиков этого сорта в мире. Потенциал урожая арабики в следующем сезоне под угрозой. @bankrollo

In [31]:
len(out["candidates"]), len(out["candidates_dedup"]), len(out["candidates_filtered"])

(150, 134, 57)

In [32]:
import numpy as np
import pandas as pd
from vllm import SamplingParams

def run_rag_hybrid_batch(
    df: pd.DataFrame,
    index,
    encoder,
    bm25,
    tokenize_fn,
    sum_model,
    sum_tokenizer,
    anchor_date: str = "2025-09-04",
    topic_queries: list[str] | None = None,
    n_topics: int = 150,
    n_random_news: int = 100,
    random_news_snip_chars: int = 1000,
    seed: int = 42,
    k_retrieve: int = 150,
    topN_each: int = 1000,
    k_docs: int = 30,
    snip_chars: int = 1500,
    max_new_tokens: int = 4000,
    max_window_days: int | None = 365,
    w_time: float = 0.5,
    w_channel: float | None = None,
    hot_window_days: int = 30,
    hot_ratio: float = 0.8,
    judge_llm=None,
    judge_tokenizer=None,
    judge_keep_threshold: int = 1,
    judge_batch_size: int = 32,
    do_dedup: bool = True,
    dedup_sim_threshold: float = 0.95,
    dedup_knn: int = 30,
    dedup_keep_per_cluster: int = 1,
    dedup_max_day_diff: int = 1,
    gen_batch_size: int = 8,
):
    rng = np.random.default_rng(seed)

    if topic_queries is None:
        if "QUERY_SPECS" in globals() and isinstance(globals()["QUERY_SPECS"], list):
            topic_queries = list(globals()["QUERY_SPECS"])
        else:
            raise ValueError("topic_queries is None and global QUERY_SPECS not found; pass topic_queries=list[str].")

    topic_queries = [q for q in topic_queries if isinstance(q, str) and q.strip()]
    topic_queries = topic_queries[:n_topics]

    date_col = "date_day" if "date_day" in df.columns else ("date" if "date" in df.columns else None)
    if date_col is None:
        raise KeyError("df must contain date_day or date column")

    eligible = df.copy()
    try:
        ad = pd.to_datetime(anchor_date, utc=True)
        dts = pd.to_datetime(eligible[date_col], errors="coerce", utc=True)
        eligible = eligible.loc[dts <= ad]
    except Exception:
        pass

    if len(eligible) == 0:
        eligible = df

    n_random_news = min(n_random_news, len(eligible))
    sample_idx = rng.choice(np.arange(len(eligible)), size=n_random_news, replace=False)
    news_sample = eligible.iloc[sample_idx].copy()

    news_queries = []
    for row in news_sample.itertuples(index=False):
        msg = getattr(row, "message", "")
        mid = getattr(row, "message_id", None)
        q = snippet(str(msg), random_news_snip_chars)
        news_queries.append((q, mid))


    items = []
    for q in topic_queries:
        items.append({"query": q, "query_type": "topic", "seed_message_id": None})
    for q, mid in news_queries:
        items.append({"query": q, "query_type": "news", "seed_message_id": mid})

    prompts = []
    prompt_row_ids = []
    rows = []

    for rid, it in enumerate(items):
        print(rid)
        query = it["query"]

        cand = hybrid_retrieve_rrf(
            df=df,
            index=index,
            encoder=encoder,
            bm25=bm25,
            tokenize_fn=tokenize_fn,
            query=query,
            k=k_retrieve,
            topN_each=topN_each,
            k_rrf=60,
            w_dense=1.0,
            w_bm25=1.0,
            anchor_date=anchor_date,
            max_window_days=max_window_days,
            w_time=w_time,
            w_channel=w_channel,
        )

        cand_before = cand
        cand_clusters = None
        members = None

        if do_dedup and cand is not None and len(cand) > 0 and encoder is not None:
            cand, cand_clusters, members = dedup_cluster_candidates_time(
                cand=cand,
                encoder=encoder,
                text_col="message",
                score_col="score_rrf",
                sim_threshold=float(dedup_sim_threshold),
                knn=int(dedup_knn),
                keep_per_cluster=int(dedup_keep_per_cluster),
                mask_numbers=False,
                max_day_diff=int(dedup_max_day_diff),
                overwrite_channel=True,
            )

        cand_after_dedup = cand

        if (
            judge_llm is not None
            and judge_tokenizer is not None
            and cand is not None
            and len(cand) > 0
        ):
            cand = judge_filter_candidates(
                cand=cand,
                query=query,
                judge_llm=judge_llm,
                judge_tokenizer=judge_tokenizer,
                keep_threshold=judge_keep_threshold,
                doc_max_chars=snip_chars,
                batch_size=judge_batch_size,
            )

        ctx = build_rag_context(
            query=query,
            cand=cand,
            anchor_date=anchor_date,
            k_docs=min(k_docs, len(cand)) if cand is not None else 0,
            snip_chars=snip_chars,
            hot_window_days=hot_window_days,
            hot_ratio=hot_ratio,
        )

        row = {
            "row_id": rid,
            "anchor_date": anchor_date,
            "query": query,
            "query_type": it["query_type"],
            "seed_message_id": it["seed_message_id"],
            "k_retrieve": k_retrieve,
            "topN_each": topN_each,
            "k_docs": k_docs,
            "snip_chars": snip_chars,
            "max_new_tokens": max_new_tokens,
            "max_window_days": max_window_days,
            "w_time": w_time,
            "w_channel": w_channel,
            "hot_window_days": hot_window_days,
            "hot_ratio": hot_ratio,
            "context": ctx,
            "summary": None,
            "n_cand_before": int(len(cand_before)) if cand_before is not None else 0,
            "n_cand_after_dedup": int(len(cand_after_dedup)) if cand_after_dedup is not None else 0,
            "n_cand_final": int(len(cand)) if cand is not None else 0,
        }
        rows.append(row)

        if cand is not None and len(cand) > 0 and sum_model is not None and sum_tokenizer is not None:
            messages = [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": ctx},
            ]
            prompt = sum_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            prompts.append(prompt)
            prompt_row_ids.append(rid)

    out_df = pd.DataFrame(rows)

    if len(prompts) == 0:
        return out_df

    sampling = SamplingParams(
        temperature=0.0,
        top_p=1.0,
        max_tokens=max_new_tokens,
    )

    summaries = [""] * len(prompts)
    for start in range(0, len(prompts), gen_batch_size):
        chunk = prompts[start:start + gen_batch_size]
        outs = sum_model.generate(chunk, sampling)
        for i, o in enumerate(outs):
            summaries[start + i] = o.outputs[0].text.strip()

    rid_to_summary = {rid: s for rid, s in zip(prompt_row_ids, summaries)}
    out_df["summary"] = out_df["row_id"].map(rid_to_summary).where(out_df["row_id"].isin(rid_to_summary.keys()), out_df["summary"])

    return out_df


In [33]:
QUERY_SPECS = [
    "Актуальный курс доллара",
    "Кофе дорожает, почему?", 
    "Почему масло дорожает?"
    "Доллар растет к рублю: причины и факты",
    "Доллар падает к рублю: причины и факты",
    "Евро к рублю: что происходит",
    "Юань к рублю: динамика и причины",
    "Причины ослабления рубля",
    "Причины укрепления рубля",
    "Прогноз курса доллара на месяц",
    "Прогноз курса доллара на квартал",
    "Прогноз курса евро на месяц",
    "Валютные интервенции: что известно",
    "Ограничения на валютные операции: изменения",
    "Валютная выручка экспортеров: требования и обсуждения",
    "Наличный доллар: дефицит и премия к курсу",
    "Спред официальный/внебиржевой курс доллара",
    "Курс рубля и нефть: как это описывают новости",
    "Курс рубля и ключевая ставка: связь в новостях",
    "Волатильность рубля: главные триггеры",
    "Риски девальвации рубля: что пишут",
    "Контроль капитала: новые меры",
    "Импорт и давление на рубль",
    "Экспорт и поддержка рубля",
    "Курс доллара выше 100 рублей: упоминания и причины",
    "Курс доллара ниже 80 рублей: упоминания и причины",
    "Курс евро выше 110 рублей: упоминания и причины",
    "Решение ЦБ по ключевой ставке: итоги и сигналы",
    "Повышение ключевой ставки: последствия",
    "Снижение ключевой ставки: последствия",
    "Инфляция ускорилась: причины",
    "Инфляция замедлилась: причины",
    "Прогноз инфляции на год: что говорят",
    "Инфляционные ожидания населения: изменения",
    "ИПЦ: свежие данные и комментарии",
    "Продовольственная инфляция: что дорожает",
    "Инфляция услуг: драйверы",
    "Индексация тарифов и инфляция",
    "Рост цен на бензин: причины и меры",
    "Рост цен на продукты: что именно",
    "Цены на овощи и фрукты: динамика",
    "Цены на мясо и молоко: динамика",
    "Цены на хлеб: динамика",
    "Цены на яйца: новая волна или спад",
    "Денежная масса и инфляция: обсуждения",
    "Кредитование и инфляция: связь в новостях",
    "Бюджетные расходы и инфляция: влияние",
    "Реальные доходы населения: динамика",
    "Зарплаты и рынок труда: динамика",
    "Безработица: изменения",
    "Потребительский спрос: рост или охлаждение",
    "Экономический рост: драйверы и оценки",
    "Деловая активность (PMI): сигналы",
    "Прогнозы Минэка и аналитиков: расхождения",
    "Дефицит бюджета: динамика",
    "Доходы бюджета: нефть/газ и ненефтегазовые",
    "Расходы бюджета: приоритеты",
    "Заимствования Минфина: планы и факты",
    "ОФЗ: спрос и доходности",
    "ФНБ/резервы: изменения",
    "Траты ФНБ: на что идут",
    "Налоги: ключевые изменения",
    "НДФЛ: изменения и обсуждения",
    "Налог на прибыль: изменения",
    "НДС: изменения и инициативы",
    "Акцизы: изменения",
    "Экспортные пошлины: изменения",
    "Импортные пошлины: изменения",
    "Пенсии: индексации и выплаты",
    "Соцвыплаты: изменения",
    "Цена нефти Brent растет: причины",
    "Цена нефти Brent падает: причины",
    "Нефть Urals: динамика",
    "ОПЕК+: решение по добыче",
    "ОПЕК+: квоты и дисциплина",
    "Потолок цен на нефть: обсуждения",
    "Экспорт нефти: изменения",
    "Экспорт нефтепродуктов: ограничения и эффекты",
    "Бензин дорожает: что происходит",
    "Дизель дорожает: что происходит",
    "Топливный демпфер: изменения",
    "Газ в Европе: цены и причины",
    "LNG рынок: новости и тренды",
    "Золото: рекорды и факторы",
    "Медь: рост/падение и причины",
    "Никель: динамика",
    "Литий: динамика",
    "Удобрения: экспорт и цены",
    "Зерно: экспортные ограничения",
    "Пшеница: мировые цены",
    "Сахар: динамика цен",
    "Какао: рост цен и причины",
    "Кофе дорожает: причины и последствия",
    "Хлопок: динамика",
    "Рынок логистики: цены и ограничения",
    "Ипотека: ставки растут",
    "Ипотека: ставки снижаются",
    "Льготная ипотека: изменения",
    "Рынок недвижимости: охлаждение или рост",
    "Потребкредиты: рост/замедление",
    "Автокредиты: динамика",
    "Просрочка по кредитам: изменения",
    "Прибыль банков: рост/падение",
    "Ограничения ЦБ для банков: меры",
    "Санкции против банков: последствия",
    "Комиссии банков: изменения",
    "Переводы и платежи: ограничения",
    "Индекс Мосбиржи растет: причины",
    "Индекс Мосбиржи падает: причины",
    "Дивиденды: крупные выплаты",
    "Отмена или снижение дивидендов: причины",
    "IPO/SPO: крупные размещения",
    "Облигации: дефолты и реструктуризации",
    "Корпоративные облигации: доходности",
    "ОФЗ: рост доходностей",
    "ОФЗ: снижение доходностей",
    "Санкции: новый пакет и влияние",
    "Параллельный импорт: изменения",
    "Экспорт в Китай: динамика",
    "Импорт из Китая: динамика",
    "Торговля с ЕС: динамика",
    "Расчеты в нацвалютах: изменения",
    "SWIFT/платежи: ограничения",
    "Цены на фрахт и логистику: динамика",
    "Торговый баланс: профицит/дефицит",
    "Снижение экспорта: причины",
    "Рост импорта: причины",
    "Торговые войны: влияние на цены сырья",
    "Риски для цепочек поставок: что пишут",
    "Автомобили дорожают: причины",
    "Автомобили дешевеют: причины",
    "Цены на новые авто: динамика",
    "Цены на б/у авто: динамика",
    "Электроника дорожает: причины",
    "Смартфоны дорожают: причины",
    "Бытовая техника дорожает: причины",
    "Одежда и обувь дорожают: причины",
    "Лекарства дорожают: причины",
    "Жилье и аренда дорожают: причины",
    "Коммунальные услуги дорожают: причины",
    "Тарифы ЖКХ: индексация",
    "Авиабилеты дорожают: причины",
    "Туризм: рост цен",
    "Ритейл: наценки и цены",
    "Продукты: скидки и акции — что происходит",
    "Шоколад дорожает: причины",
    "Чай дорожает: причины",
    "Молоко дорожает: причины",
    "Рыба дорожает: причины",
    "Стройматериалы дорожают: причины",
    "Крупная сделка в отрасли: последствия",
    "Слияния и поглощения в ритейле: новости",
    "Продажа сети кофеен: причины и эффект",
    "Финансовые результаты компаний: ключевые сюжеты",
    "Компания подняла цены: причины",
    "Компания сократила производство: причины",
    "Компания расширяет производство: причины",
    "Дефицит кадров в отрасли: факты",
    "Рост зарплат в отрасли: факты",
    "Новые инвестиционные проекты: где и какие",
    "Регулирование отрасли: изменения",
    "Биткоин растет: причины",
    "Биткоин падает: причины",
    "Регулирование криптовалют: новости",
    "Майнинг: регулирование и тарифы",
    "Цифровой рубль: новости и пилоты",
    "Налоги на крипто: обсуждения",
    "Мошенничество в крипте: случаи",
    "Платежи в крипте: ограничения",
]

In [34]:
len(QUERY_SPECS)

168

In [ ]:
eval_df = run_rag_hybrid_batch(
    df=df,
    index=index,
    encoder=encoder,
    bm25=bm25,
    tokenize_fn=tokenize_ru,
    sum_model=model,
    sum_tokenizer=tokenizer,
    judge_llm=model,
    judge_tokenizer=tokenizer,
    anchor_date="2025-09-04",
    topic_queries=QUERY_SPECS,
    n_topics=168,
    n_random_news=32,
    gen_batch_size=8,
    max_new_tokens=2000,
    k_docs=30,
    snip_chars=800,
    max_window_days=365,
    w_time=0.6,
    w_channel=None,
    hot_window_days=30,
    hot_ratio=0.75,
)

eval_df.to_parquet("eval_df_temporalrag.parquet", index=False)

In [36]:
eval_df

,row_id,anchor_date,query,query_type,seed_message_id,k_retrieve,topN_each,k_docs,snip_chars,max_new_tokens,max_window_days,w_time,w_channel,hot_window_days,hot_ratio,context,summary,n_cand_before,n_cand_after_dedup,n_cand_final
0,0,2025-09-04,Актуальный курс доллара,topic,None,150,1000,30,800,2000,365,0.6,None,30,0.75,АКТУАЛЬНАЯ ДАТА ОБЗОРА: 2025-09-04\nВОПРОС/ЗАП...,### 1) Запрос и актуальная дата\n* Запрос: Акт...,150,124,122
1,1,2025-09-04,"Кофе дорожает, почему?",topic,None,150,1000,30,800,2000,365,0.6,None,30,0.75,АКТУАЛЬНАЯ ДАТА ОБЗОРА: 2025-09-04\nВОПРОС/ЗАП...,### 1) Запрос и актуальная дата\n* Запрос: Коф...,150,136,47
2,2,2025-09-04,Почему масло дорожает?Доллар растет к рублю: п...,topic,None,150,1000,30,800,2000,365,0.6,None,30,0.75,АКТУАЛЬНАЯ ДАТА ОБЗОРА: 2025-09-04\nВОПРОС/ЗАП...,### 1) Запрос и актуальная дата\n* Запрос: Поч...,150,129,87
3,3,2025-09-04,Доллар падает к рублю: причины и факты,topic,None,150,1000,30,800,2000,365,0.6,None,30,0.75,АКТУАЛЬНАЯ ДАТА ОБЗОРА: 2025-09-04\nВОПРОС/ЗАП...,### 1) Запрос и актуальная дата\n* Запрос: Дол...,150,130,86
4,4,2025-09-04,Евро к рублю: что происходит,topic,None,150,1000,30,800,2000,365,0.6,None,30,0.75,АКТУАЛЬНАЯ ДАТА ОБЗОРА: 2025-09-04\nВОПРОС/ЗАП...,### 1) Запрос и актуальная дата\n* Запрос: Евр...,150,129,96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,2025-09-04,Власти США заранее располагали информацией о т...,news,209736,150,1000,30,800,2000,365,0.6,None,30,0.75,АКТУАЛЬНАЯ ДАТА ОБЗОРА: 2025-09-04\nВОПРОС/ЗАП...,### 1) Запрос и актуальная дата\n* Запрос: Вла...,150,138,55
196,196,2025-09-04,Израиль сегодня - почти русскоговорящая страна...,news,300858,150,1000,30,800,2000,365,0.6,None,30,0.75,АКТУАЛЬНАЯ ДАТА ОБЗОРА: 2025-09-04\nВОПРОС/ЗАП...,### 1) Запрос и актуальная дата\n* Запрос: Изр...,150,145,6
197,197,2025-09-04,"Конституционный суд постановил, что дети, зача...",news,280358,150,1000,30,800,2000,365,0.6,None,30,0.75,АКТУАЛЬНАЯ ДАТА ОБЗОРА: 2025-09-04\nВОПРОС/ЗАП...,### 1) Запрос и актуальная дата\n* Запрос: Кон...,150,142,4
198,198,2025-09-04,"Ми-171, перевозивший президента Ирана Ибрахима...",news,1444,150,1000,30,800,2000,365,0.6,None,30,0.75,АКТУАЛЬНАЯ ДАТА ОБЗОРА: 2025-09-04\nВОПРОС/ЗАП...,### 1) Запрос и актуальная дата\n* Запрос: Ми-...,150,146,2
